# 第5章 再び，Deep Learning！

## フレームワーク(TensorFlow + Keras)を使ってXORを学習してみよう！
ここまで見てきたように，頑張れば自力でニューラルネットワークを実装することができる．  
しかし，とても大変だったことがわかると思う．  
さらに，今，Deep Learningの研究を行っている人が多数いる．  
その人達と成果物を共有しようとするとなると，各自が自作したプログラムでは議論が難しい．  

そこで，Deep Learningを動かす枠組み（フレームワーク）が多く開発されている．  
ここでは，それらの一つであるGoogleが開発したTensorFlowを紹介する．  
TensorFlowはニューラルネットワークを動かすというよりは，テンソル（3次元以上の多次元行列）の演算を行うために開発されたものである．  
もちろん，Deep Learningのために開発されたのであるが，ニューラルネットワークは行列の演算を多く行う．  
そのため，行列演算のために，しかも並列計算が可能になるように開発されたものがTensorFlowである．  

さらに，　KerasはTensorFlowのフロントエンドであり，Deep Learningのモデルを短い記述で実装できる．  
一般的に，TensorFlowでDeep Learningを実装しようとすると，Kerasを用いる．  

TensorFlow + Kerasを使えば簡単に実装できる．  
ニューラルネットワークの構造を変えたり，活性化関数を変更したり，最適化のアルゴリズムを変更したり，とにかくいろいろできる．  
さらに，グラフを表示したり，ネットワークの出力を可視化したり，とにかく機能が豊富である．  

講義の最初に動作させた画像認識のプログラムを例に説明する．

In [3]:
# TensorFlow をセットアップする
import tensorflow as tf
import numpy as np

# トレーニングデータ
# XOR
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [0]])

# モデル設定（階層型）
model = tf.keras.Sequential()

# 入力層 - 隠れ層（入力2，中間2）
model.add(tf.keras.layers.Dense(input_dim=2, units=2))
model.add(tf.keras.layers.Activation('sigmoid'))

# 隠れ層 - 出力層（出力1）
model.add(tf.keras.layers.Dense(units=1))
model.add(tf.keras.layers.Activation('linear'))

# 最適化 stochastic gradient descent
sgd = tf.keras.optimizers.SGD(learning_rate=0.1)
model.compile(loss='mean_squared_error', optimizer=sgd)

# モデル学習
model.fit(X, Y, epochs=4000, verbose=0)

# 学習結果の確認
print(model.predict(X))

2023-02-28 14:01:16.658995: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 24ms/step
[[9.0599060e-06]
 [9.9999309e-01]
 [9.9999452e-01]
 [3.7550926e-06]]


2023-02-28 14:01:33.450707: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


重みを確認する．

In [ ]:
model.get_weights()

ベクトル・行列演算で計算してみる．

XORの最初のパターン[0,0]を入力して，重みとの積を計算する．

In [ ]:
p=0
a = np.dot(model.get_weights()[0].T,X[p])
print(a)

それに閾値を足す．

In [ ]:
b = a+model.get_weights()[1]
print(b)

シグモイド関数に通す． これで中間層2個の出力値を得る．

In [ ]:
c = tf.math.sigmoid(b)
print(b)

同様に出力層の出力を計算する．

In [ ]:
out = np.dot(model.get_weights()[2].T,c)+model.get_weights()[3]
print(out)

これでXORの1番目のパターン[0,0]を入力して，0．0が出力されていることがわかる．  
フレームワークTensowFlowを用いても，やっている計算は同じであることがわかる．  

どうように4パターンを入力し，そのときに予測を出力する．

In [ ]:
for p in range(4):
    out1 = tf.math.sigmoid(np.dot(model.get_weights()[0].T,X[p])+model.get_weights()[1])
    out2 = np.dot(model.get_weights()[2].T,out1)+model.get_weights()[3]
    print(out2)

## ニューラルネットワークの基礎理論を理解した上で，もう一度，文字認識をやってみよう．

### 意味を理解しながら，一つずつ動かしてみよう

In [ ]:
# TensorFlow をセットアップする
import tensorflow as tf

学習データであるMNISTデータを読み込む．  
もともとの文字のデータは0から255のグレースケールだが，255.0で割ることで，0．0から1．0に変換している．

In [ ]:
# MNIST（0から9の数字画像）セットを読み込む
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

学習データを眺めてみる．  
トレーニングデータは60000個，テストデータは10000個である．

In [ ]:
print(len(x_train))
print(len(x_test))

次にデータの中身を覗いてみよう．  


In [ ]:
x_train

これではさっぱりわからいので，トレーニングデータの最初の文字を表示してみよう．  
また，画像で表示してみると「5」に見える．  

In [ ]:
for i in x_train[0]:
    for j in i:
        print('{:.1f}'.format(j), end=' ')
    print()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0] * 255, cmap="gray_r")
plt.show()

# 一般的にグレースケールは0が黒，255が白だが，反転している．

画像サイズは28x28である．  
これを1列に並べて，784次元の特徴ベクトルとして階層型ニューラルネットワークで学習させる．

In [ ]:
import numpy as np
np.shape(x_train[0])

### 機械学習モデルを構築する
Flattenは縦と横の画像を1列に並べる操作である．  
Dropoutは学習中に20%の重みをランダムに削除して学習させている．  
これにより汎化性能の向上が期待される．  
最終的には0から9の10種の数字なので，出力層は10個のニューロンを持つことになる．

In [ ]:
# 機械学習モデルを構築する
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

### モデルの構成

損失関数はクロスエントロピーを用いる．  
0から9の数字を予測するため，出力層10個の各ニューロンの出力値が，0.0から1.0の確率値に対応している．  
最適化はadamを用いる． (参考文献:[An overview of gradient descent optimization algorithms](https://ruder.io/optimizing-gradient-descent/))  

In [ ]:
# モデルの構成
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### 学習

ここまででニューラルネットワークのモデル構築ができた．  
あとはバックプロパゲーションで学習させるだけである．  

In [ ]:
# 学習
model.fit(x_train, y_train, epochs=5)

### モデルの評価

学習には用いなかったテストデータでモデルの評価を行う．

In [ ]:
# モデルの評価を行う
model.evaluate(x_test,  y_test, verbose=2)

### テストデータに対する予測を行う

テストデータを入力したときに出力層10個のニューロンの値をみてみよう．  
それぞれ確率の値となっているので，最大値のニューロンに該当する数字を予測した数字とする．  
先頭5個のテストデータを入力して確認してみよう．

In [ ]:
# テストデータに対する予測を行う
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[:5])

## 文字認識の結果を確認してみよう

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

row = 3
col = 4
predictions = probability_model(x_test[:row*col])

fig = plt.figure(figsize=(row, col))

# 予測と正解を可視化して比べてみる
fig.subplots_adjust(left=0, right=2, bottom=0.5, top=2, hspace=0.3, wspace=0.3)
for index, prediction in enumerate(predictions):
    ax = fig.add_subplot(row, col, index + 1)
    ax.set_title("prediction:"+str(np.argmax(prediction)))
    plt.imshow(x_test[index] * 255, cmap="gray_r")
plt.show()

### 参考文献
- [初心者のための TensorFlow 2.0 入門](https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ja)
- [TensorFlowのチュートリアルをコメント付けながら実行してみた（初心者のための_TensorFlow_2_0_入門）](https://qiita.com/penpenta/items/ee45f58d416c656639aa)

# この後どうするか？
- とにかく自分で動かしてみる．
- 数学で表現されている基礎理論を理解する．
- 実際に別の課題に取り組んで見る．
- 1番大変なのは前処理！
- 2番目に大変なのは良質のデータを大量に集めること！
- 書籍やネットに多くの情報があるので取り組みやすい状況ではあるが，とにかく動かしてみることが一番大事

[講義資料に戻る](https://github.com/crotsu/Deep_Learning_Starting_with_Examples)

このページは[このリンク](https://colab.research.google.com/github/crotsu/Deep_Learning_Starting_with_Examples/blob/main/chap5_deeplearning/chap5_deeplearning.ipynb)よりgoogle colaboratoryで動作させることができる．  